In [ ]:
import numpy as np
import pandas as pd
import datetime
from tqdm.notebook import tqdm
from random import sample
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn import tree
from sklearn.metrics import roc_auc_score
import random


In [ ]:
random.seed(10) 

from functions import *
# Load data - set index column, decimal point, separator
data = pd.read_csv('hw1_devsample.csv', sep=',',
                   decimal='.', index_col='SK_ID_CURR')

# print time of data being loaded - use strftime
print(f'Data loaded on:   {datetime.datetime.now().strftime(format="%Y-%m-%d %H:%M:%S")}')
data_cart = data.copy()

data_test = pd.read_csv('hw1_outofsample.csv', sep=',',decimal='.', index_col='SK_ID_CURR')

In [ ]:
from sklearn.pipeline import Pipeline
import category_encoders as ce

In [ ]:
target = data_cart['TARGET']

In [ ]:
data_cart = data_cart.drop(['MONTH','TIME','DAY','BASE','TARGET'], axis=1)

In [ ]:
data_cart.info()

In [ ]:
cols_pred = data_cart.columns

In [ ]:
cols_pred_cat = [col for col in cols_pred if data_cart[col].dtype == 'O']

In [ ]:
cols_pred_cat

In [ ]:
data_cart_2 = ce.target_encoder.TargetEncoder(verbose=0, 
cols=cols_pred_cat,
#drop_invariant=False, 
return_df=True, handle_missing='value', 
handle_unknown='value', min_samples_leaf=1
#, smoothing=1.0
)

In [ ]:
data_cart2 = data_cart_2.fit_transform(data_cart,target)

In [ ]:
data_cart2.info()

In [ ]:
data_cart2[data_cart2.columns].isna().sum()

In [ ]:
pipe = Pipeline([
    ('select', SelectKBest()),
    ('model', tree)
])

In [ ]:
data_saving = pd.DataFrame({'SK_ID_CURR' : data_test.index ,'prediction' : prediction})
data_saving.to_csv('DS2_22_HW1_CADIOU&PAIN.csv',index=False,sep=',')
data_saving.info()